In [32]:
import os
from glob import glob

import pandas as pd
import numpy as np
from collections import Counter

import pickle
import gc

from tqdm import tqdm
tqdm.pandas()

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [33]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = './models'
UTILS = './utils'
SUBM = './submissions'

### Correction if client never used predicted function

In [34]:
load_col_for_check = ['client_pin', 'timestamp']
data_corr = pd.read_csv(os.path.join(DATA_OWN, 'data_pred.csv'),  usecols=load_col_for_check, parse_dates=['timestamp'])

In [35]:
subm = pd.read_csv(os.path.join(SUBM, 'subm_client_6diff_lt_relat_mlp.csv'))

In [36]:
#for idx in data.index:
#    if data.loc[0].client_pin != subm.loc[0].client_pin:
#       print('What?')

In [37]:
client_used_target_df = pd.read_csv(os.path.join(DATA_OWN, 'client_target_time_mean.csv'))
client_used_target_df.drop('Unnamed: 0', axis = 1, inplace = True)
client_used_target_df.tail()

,client_pin,mean_main_screen,mean_statement,mean_credit_info,mean_own_transfer,mean_mobile_recharge,mean_phone_money_transfer,mean_card2card_transfer,mean_chat,mean_card_recharge,mean_invest
79263,f2c4e22ecd806d691e24fa00fded6d9d,2.222591,0.0,0.0,0.0,0.0,4.810209,0.0,0.0000,0.0000,0.0000
79264,e01e7a15cec5168d0eee370c70851f38,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,13.1461
79265,d1007b6356164d9150b57f35e42b7812,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,3.1257,0.0000,0.0000
79266,9193bff6506617d47c92d3029807ea2a,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,1.1089,0.0000
79267,1021464b1a37f1f1c1bff1cc9dd24cf1,0.203350,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.0000


In [38]:
last_target = pd.read_csv(os.path.join(DATA_OWN, 'last_target_begore.csv'), parse_dates=['timestamp'])
last_target.drop('Unnamed: 0', axis = 1, inplace = True)
last_target.head()

,client_pin,timestamp,last_target_begore
0,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-02 22:31:04,first_appear
1,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-19 18:37:26,statement
2,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-22 23:18:45,main_screen
3,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-27 17:29:57,main_screen
4,000033b6509acd1c8eb0d06ebd2e1de9,2020-03-04 22:39:45,main_screen


In [39]:
#data_check['predicted'] = pred_sgd_full
data_corr = data_corr.merge(subm, how='left', on='client_pin', validate='one_to_one')
data_corr = data_corr.merge(last_target, how='left', on=['client_pin', 'timestamp'], validate='one_to_one')
data_corr = data_corr.merge(client_used_target_df, how='left', on='client_pin', validate='many_to_one')

In [40]:
data_corr.head()

,client_pin,timestamp,prediction,last_target_begore,mean_main_screen,mean_statement,mean_credit_info,mean_own_transfer,mean_mobile_recharge,mean_phone_money_transfer,mean_card2card_transfer,mean_chat,mean_card_recharge,mean_invest
0,f0c674b2bb4dc64be607029271d706ec,2020-08-01 00:00:53,credit_info,credit_info,0.000000,0.000000,138.763740,0.000000,0.000000,0.000000,0.00000,0.000000,229.9785,0.0
1,90725b54ce77576883813d87749df6bd,2020-08-01 00:02:57,statement,main_screen,704.800725,352.050050,1194.990009,1501.724528,903.154519,1000.167013,0.00000,1325.883169,0.0000,0.0
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,2020-08-01 00:03:14,main_screen,main_screen,337.308083,442.294533,126.567058,446.234258,0.000000,0.000000,0.00000,520.799683,0.0000,0.0
3,831bf4c0ecccc55e536b8cfb9153d672,2020-08-01 00:04:44,statement,statement,0.000000,577.283240,1275.399950,0.000000,0.000000,0.000000,1303.23885,0.000000,0.0000,0.0
4,3f1a5a1c492ce877af833113e59c5797,2020-08-01 00:06:40,main_screen,main_screen,372.908882,420.156229,0.000000,0.000000,476.626439,0.000000,0.00000,692.842282,0.0000,0.0


In [41]:
def correct_low_info_client_predict(inp_row):
    
    global n_corrected
    
    if inp_row['mean_' + inp_row.prediction] == 0:
        n_corrected += 1
        return inp_row.last_target_begore
    else:
        return inp_row.prediction

In [42]:
def check_client_low_info(inp_data):
    
    inp_data['corr_prediction'] = inp_data.progress_apply(correct_low_info_client_predict, axis = 1)
    
    return inp_data.corr_prediction

In [43]:
%%time

n_corrected = 0
pred_new = check_client_low_info(data_corr)
print(n_corrected)

100%|█████████████████████████████████████████████████████████████████████████| 79268/79268 [00:01<00:00, 41925.90it/s]

98
Wall time: 1.9 s


In [45]:
subm.prediction = pred_new

In [46]:
subm.to_csv(os.path.join(SUBM, 'subm_client_6diff_lt_relat_corr_mlp.csv'), index = False)